# Imports and Pre-requisites

In [ ]:
import json
import ee
import geemap
from dateutil.utils import today
from geemap import geojson_to_ee, ee_to_geojson

In [ ]:
ee.Authenticate()

In [ ]:
ee.Initialize(project="rwanda-climate-alerts")

# Pretty print a JSON file

In [ ]:
sector_boundaries_path = "../data/sector_boundaries/geojson/Sector_Boundaries.geojson"
district_boundaries_path = "../data/district_boundaries/geojson/District_Boundaries.geojson"

def pretty_print_json(file_path):
    with open(file_path, "r") as infile:
        data = json.load(infile)

    with open(file_path, "w") as outfile:
        json.dump(data, outfile, indent=None)

# pretty_print_json(sector_boundaries_path)
# pretty_print_json(district_boundaries_path)

# Import GeoJSONs

In [ ]:
m = geemap.Map()

## Load GeoJSON as python dict (didn't work)

In [ ]:
sector_boundaries_path = "../data/sector_boundaries/geojson/Sector_Boundaries.geojson"
district_boundaries_path = "../data/district_boundaries/geojson/District_Boundaries.geojson"

with open(sector_boundaries_path, "r") as f:
    sector_boundaries = json.load(f)

with open(district_boundaries_path, "r") as f:
    district_boundaries = json.load(f)

print(type(sector_boundaries))

## Use ee.FeatureCollection("{file_path}") (did work)

In [ ]:
district_boundaries_ee = ee.FeatureCollection(district_boundaries_path)
sector_boundaries_ee = ee.FeatureCollection(sector_boundaries_path)

# Note:
# ee.FeatureCollection() works with file paths

In [ ]:
print(district_boundaries_ee.getInfo()) # Not working

In [ ]:
print(type(sector_boundaries_ee))

## Attempt to load info from Earth Engine API

### Use district_boundaries_ee.filter(ee.Filter.eq("district_id", "{id}"))

In [ ]:
# Load GeoJSON as a FeatureCollection
# rwanda = ee.FeatureCollection(sector_boundaries_ee)

In [ ]:
# Or a specific sector (sector_id = 5704)
mareba = sector_boundaries_ee.filter(ee.Filter.eq("sector_id", "5704"))

geometry = mareba.geometry()

In [ ]:
# Example: Get Bugesera District (district_id = 57)
bugesera = (
    district_boundaries_ee
    .filter(ee.Filter.eq("district_id", "57"))
)

print(type(bugesera))

In [ ]:
print(bugesera)

In [ ]:
# Extract the geometry
geometry = bugesera.geometry()

print(type(geometry))

In [ ]:
# Not working
# Probable cause:EE doesn't have data for the very specific geometry
print(geometry.getInfo())

### Failed: Try to get area from EE

In [ ]:
# Print info
# print("Bugesera sector geometry:", geometry.getInfo())

# Example: calculate its area in square kilometers
area_km2 = geometry.area().divide(1e6).getInfo()
print("Bugesera sector area (km²):", area_km2)

## Add district to map

In [ ]:
m.addLayer(geometry, {'color': 'black'}, 'Geometry [black: geometry')

# Test ShapeFile

In [ ]:
district_boundaries_path = "projects/rwanda-climate-alerts/assets/rwanda-district-boundaries"
sector_boundaries_path = "projects/rwanda-climate-alerts/assets/rwanda-sector-boundaries"

district_boundaries_ee = ee.FeatureCollection(district_boundaries_path)
sector_boundaries_ee = ee.FeatureCollection(sector_boundaries_path)

print(type(district_boundaries_ee))

In [ ]:
Gasabo = (
    district_boundaries_ee
    .filter(ee.Filter.eq("district", "Gasabo"))
)

print(type(Gasabo))

In [ ]:
geometry = Gasabo.geometry()

# print(geometry.getInfo())

In [ ]:
area_km2 = geometry.area().divide(1e6).getInfo()
print("Bugesera sector area (km²):", area_km2)

In [ ]:
m.addLayer(geometry, {'color': 'black'}, 'Geometry [black]: geometry')

## Attempt 2: add shapefile to map

In [ ]:
district_boundaries_shp = "../data/district_boundaries/shapefile/District_Boundaries.shp"
sector_boundaries_shp = "../data/sector_boundaries/shapefile/Sector_Boundaries.shp"

districts = geemap.shp_to_ee(district_boundaries_shp)
m.addLayer(districts, {'color': 'black'}, 'Geometry [black]: districts')

# Test GeoJson

In [ ]:
# Define a Geometry object.
geometry = ee.Geometry({
    'type': 'Polygon',
    'coordinates': [[
        [-122.081, 37.417],
        [-122.086, 37.421],
        [-122.084, 37.418],
        [-122.089, 37.416],
    ]],
})

# Apply the toGeoJSONString method to the Geometry object.
geometry_to_geojson_string = geometry.toGeoJSONString()

# Print the result.
display('geometry.toGeoJSONString(...) =', geometry_to_geojson_string)

# Display relevant geometries on the map.
m = geemap.Map()
m.set_center(-122.085, 37.422, 15)
m.add_layer(geometry, {'color': 'black'}, 'Geometry [black]: geometry')

# Collect Datasets

## Rainfall

In [ ]:
from datetime import date, timedelta

today = date.today()
print(type(today))

In [ ]:
from datetime import datetime

# Example 1: Basic YYYY-MM-DD format
date_string_1 = "2023-10-30"
date_format_1 = "%Y-%m-%d"
date_object_1 = datetime.strptime(date_string_1, date_format_1)
print(f"Date object 1: {date_object_1}")
print(f"Type of date_object_1: {type(date_object_1)}")

# Final Map

In [ ]:
m = geemap.Map(center=[-2.0, 30.0], zoom=7)
m